# 4.8 Grouping Data & Aggregating Variables

## This script contains the following points:

### 01. Importing Libraries
### 02. Importing Data
### 03. Checking Dataframes
### 04. Finding the Aggregated Mean
### 05. Creating a Loyalty Flag
### 06. Creating a Spending Flag
### 07. Creating a Frequency Flag
### 08. Exporting Data

## 01. Importing Libraries

In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import os

## 02. Importing Data

In [2]:
# Set project folder as a string
path = r'/Users/matthewjones/Documents/CareerFoundry/Data Immersion/Achievement 4/04-2024 Instacart Basket Analysis'

In [3]:
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_merged_NewVariables.pkl'))

## 03. Checking Dataframes

In [4]:
# Check the shape and output of the dataframe
ords_prods_merge.shape

(32404859, 18)

In [5]:
ords_prods_merge.head(10)

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order,new_customer,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_label,busiest_day,busiest_days,busiest_period_of_day
0,2539329,1,1,2,8,NaN,True,196,1,0,Soda,77,7,9.0,Mid-Range Product,Regularly busy,Regularly busy,Average Orders
1,2539329,1,1,2,8,NaN,True,14084,2,0,Organic Unsweetened Vanilla Almond Milk,91,16,12.5,Mid-Range Product,Regularly busy,Regularly busy,Average Orders
2,2539329,1,1,2,8,NaN,True,12427,3,0,Original Beef Jerky,23,19,4.4,Low-Range Product,Regularly busy,Regularly busy,Average Orders
3,2539329,1,1,2,8,NaN,True,26088,4,0,Aged White Cheddar Popcorn,23,19,4.7,Low-Range Product,Regularly busy,Regularly busy,Average Orders
4,2539329,1,1,2,8,NaN,True,26405,5,0,XL Pick-A-Size Paper Towel Rolls,54,17,1.0,Low-Range Product,Regularly busy,Regularly busy,Average Orders
5,2398795,1,2,3,7,15.0,False,196,1,1,Soda,77,7,9.0,Mid-Range Product,Regularly busy,Least busy,Average Orders
6,2398795,1,2,3,7,15.0,False,10258,2,0,Pistachios,117,19,3.0,Low-Range Product,Regularly busy,Least busy,Average Orders
7,2398795,1,2,3,7,15.0,False,12427,3,1,Original Beef Jerky,23,19,4.4,Low-Range Product,Regularly busy,Least busy,Average Orders
8,2398795,1,2,3,7,15.0,False,13176,4,0,Bag of Organic Bananas,24,4,10.3,Mid-Range Product,Regularly busy,Least busy,Average Orders
9,2398795,1,2,3,7,15.0,False,26088,5,1,Aged White Cheddar Popcorn,23,19,4.7,Low-Range Product,Regularly busy,Least busy,Average Orders


## 04. Finding the Aggregated Mean

In [6]:
# Find the aggregated mean of the order_number, grouped by department_id
ords_prods_merge.groupby('department_id')['order_number'].mean()

department_id
1     15.457838
2     17.277920
3     17.170395
4     17.811403
5     15.215751
6     16.439806
7     17.225802
8     15.340650
9     15.895474
10    20.197148
11    16.170638
12    15.887671
13    16.583536
14    16.773669
15    16.165037
16    17.665606
17    15.694469
18    19.310397
19    17.177343
20    16.473447
21    22.902379
Name: order_number, dtype: float64

##### The aggregated mean of order_number in the total dataframe is roughly equivalent to the aggregate mean in the subset.
##### This tells me that the subset dataframe may be a usable sample population for the entire dataframe.

## 05. Creating a Loyalty Flag

### Apply the transform() function on the order_number column to get the maximum orders for each user

In [7]:
# Create a new column (max_order) of the maximum orders for each user
ords_prods_merge['max_order'] = ords_prods_merge.groupby('user_id')['order_number'].transform(np.max)

/var/folders/40/hn3x1h6s6vgb3bv80rtm46q80000gn/T/ipykernel_65110/1192305582.py:2: FutureWarning: The provided callable <function max at 0x104553920> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  ords_prods_merge['max_order'] = ords_prods_merge.groupby('user_id')['order_number'].transform(np.max)


In [8]:
# Check the output of the max_order column
ords_prods_merge['max_order'].value_counts(dropna = False)

max_order
99    1171333
8      811843
6      811396
9      810213
7      803979
       ...   
97      44949
98      44587
96      40453
2           6
1           5
Name: count, Length: 99, dtype: int64

### Assign the Loyalty Flag to each customer based on their maximum order

In [9]:
# Assign 'Loyal Customer' to customers with a max_order of more than 40
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal Customer'

In [10]:
# Assign 'Regular Customer' to customers with a max_order of more than 10 and less than or equal to 40
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular Customer'

In [11]:
# Assign 'New Customer' to customers with a max_order of less than or equal to 10
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New Customer'

In [12]:
# Check the output of the loyalty_flag column
ords_prods_merge['loyalty_flag'].value_counts(dropna = False)

loyalty_flag
Regular Customer    15876776
Loyal Customer      10284093
New Customer         6243990
Name: count, dtype: int64

##### There are more Regular Customers than New or Loyal Customers

### Determine the spending habits of the different customer loyalty tiers

In [13]:
# Find the aggregate mean, minimum, and maximum for the prices variable, grouped by the loyalty_flag groups
ords_prods_merge.groupby('loyalty_flag').agg({'prices' : ['mean', 'min', 'max']})

prices           
                      mean  min   max
loyalty_flag                         
Loyal Customer    7.773575  1.0  25.0
New Customer      7.801206  1.0  25.0
Regular Customer  7.798262  1.0  25.0

##### New Customers purchase more expensive items on average than Regular Customers, and Regular Customers purchase more expensive items than Loyal Customers

##### We could look into offering new customer deals that incentivize purchasing expensive items on their first orders, since these customers are already prone to that behavior

## 06. Creating a Spending Flag

### Apply the transform() function on the prices column to get the average prices of items across all orders for each user

In [14]:
# Create a new column (average_price_across_orders) of the average price of items purchased by each user
ords_prods_merge['average_price_across_orders'] = ords_prods_merge.groupby('user_id')['prices'].transform(np.mean)

/var/folders/40/hn3x1h6s6vgb3bv80rtm46q80000gn/T/ipykernel_65110/1536517993.py:2: FutureWarning: The provided callable <function mean at 0x104564360> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  ords_prods_merge['average_price_across_orders'] = ords_prods_merge.groupby('user_id')['prices'].transform(np.mean)


In [15]:
# Check to see that new column was created successfully
ords_prods_merge.head(10)

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order,new_customer,product_id,add_to_cart_order,reordered,...,aisle_id,department_id,prices,price_label,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,average_price_across_orders
0,2539329,1,1,2,8,NaN,True,196,1,0,...,77,7,9.0,Mid-Range Product,Regularly busy,Regularly busy,Average Orders,10,New Customer,6.367797
1,2539329,1,1,2,8,NaN,True,14084,2,0,...,91,16,12.5,Mid-Range Product,Regularly busy,Regularly busy,Average Orders,10,New Customer,6.367797
2,2539329,1,1,2,8,NaN,True,12427,3,0,...,23,19,4.4,Low-Range Product,Regularly busy,Regularly busy,Average Orders,10,New Customer,6.367797
3,2539329,1,1,2,8,NaN,True,26088,4,0,...,23,19,4.7,Low-Range Product,Regularly busy,Regularly busy,Average Orders,10,New Customer,6.367797
4,2539329,1,1,2,8,NaN,True,26405,5,0,...,54,17,1.0,Low-Range Product,Regularly busy,Regularly busy,Average Orders,10,New Customer,6.367797
5,2398795,1,2,3,7,15.0,False,196,1,1,...,77,7,9.0,Mid-Range Product,Regularly busy,Least busy,Average Orders,10,New Customer,6.367797
6,2398795,1,2,3,7,15.0,False,10258,2,0,...,117,19,3.0,Low-Range Product,Regularly busy,Least busy,Average Orders,10,New Customer,6.367797
7,2398795,1,2,3,7,15.0,False,12427,3,1,...,23,19,4.4,Low-Range Product,Regularly busy,Least busy,Average Orders,10,New Customer,6.367797
8,2398795,1,2,3,7,15.0,False,13176,4,0,...,24,4,10.3,Mid-Range Product,Regularly busy,Least busy,Average Orders,10,New Customer,6.367797
9,2398795,1,2,3,7,15.0,False,26088,5,1,...,23,19,4.7,Low-Range Product,Regularly busy,Least busy,Average Orders,10,New Customer,6.367797


In [16]:
# Check the output of the average_price_across_orders column
ords_prods_merge['average_price_across_orders'].value_counts(dropna = False)

average_price_across_orders
8.000000     10492
7.500000      9116
8.100000      7685
7.750000      7435
7.400000      7030
             ...  
13.133333        3
3.300000         3
2.700000         3
6.400000         3
3.533333         3
Name: count, Length: 128082, dtype: int64

### Assign the Spending Flag to each customer based on their average item price

In [17]:
# Assign 'Low Spender' to customers with a average item price of less than 10
ords_prods_merge.loc[ords_prods_merge['average_price_across_orders'] < 10, 'spending_flag'] = 'Low Spender'

In [18]:
# Assign 'High Spender' to customers with a average item price of 10 or greater
ords_prods_merge.loc[ords_prods_merge['average_price_across_orders'] >= 10, 'spending_flag'] = 'High Spender'

In [19]:
# Check the frequency of the spending_flag column
ords_prods_merge['spending_flag'].value_counts(dropna = False)

spending_flag
Low Spender     32285131
High Spender      119728
Name: count, dtype: int64

##### There are many more Low Spenders than there are High Spenders
##### The marketing team should target the High Spenders to get them to try other higher priced items, or discount higher priced items to the Low Spenders to get them interested in purchasing these items regularly

## 07. Creating a Frequency Flag

### Apply the transform() function on the days_since_prior_order column to get the median days since prior order for each user

In [20]:
# Create a new column (medain_days_since_prior_order) of the regularity of each user's ordering behavior
ords_prods_merge['median_days_since_prior_order'] = ords_prods_merge.groupby('user_id')['days_since_prior_order'].transform(np.median)

/var/folders/40/hn3x1h6s6vgb3bv80rtm46q80000gn/T/ipykernel_65110/3171421059.py:2: FutureWarning: The provided callable <function median at 0x1046d0400> is currently using SeriesGroupBy.median. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "median" instead.
  ords_prods_merge['median_days_since_prior_order'] = ords_prods_merge.groupby('user_id')['days_since_prior_order'].transform(np.median)


In [21]:
# Check to see that new column was created successfully
ords_prods_merge.head(10)

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order,new_customer,product_id,add_to_cart_order,reordered,...,prices,price_label,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,average_price_across_orders,spending_flag,median_days_since_prior_order
0,2539329,1,1,2,8,NaN,True,196,1,0,...,9.0,Mid-Range Product,Regularly busy,Regularly busy,Average Orders,10,New Customer,6.367797,Low Spender,20.5
1,2539329,1,1,2,8,NaN,True,14084,2,0,...,12.5,Mid-Range Product,Regularly busy,Regularly busy,Average Orders,10,New Customer,6.367797,Low Spender,20.5
2,2539329,1,1,2,8,NaN,True,12427,3,0,...,4.4,Low-Range Product,Regularly busy,Regularly busy,Average Orders,10,New Customer,6.367797,Low Spender,20.5
3,2539329,1,1,2,8,NaN,True,26088,4,0,...,4.7,Low-Range Product,Regularly busy,Regularly busy,Average Orders,10,New Customer,6.367797,Low Spender,20.5
4,2539329,1,1,2,8,NaN,True,26405,5,0,...,1.0,Low-Range Product,Regularly busy,Regularly busy,Average Orders,10,New Customer,6.367797,Low Spender,20.5
5,2398795,1,2,3,7,15.0,False,196,1,1,...,9.0,Mid-Range Product,Regularly busy,Least busy,Average Orders,10,New Customer,6.367797,Low Spender,20.5
6,2398795,1,2,3,7,15.0,False,10258,2,0,...,3.0,Low-Range Product,Regularly busy,Least busy,Average Orders,10,New Customer,6.367797,Low Spender,20.5
7,2398795,1,2,3,7,15.0,False,12427,3,1,...,4.4,Low-Range Product,Regularly busy,Least busy,Average Orders,10,New Customer,6.367797,Low Spender,20.5
8,2398795,1,2,3,7,15.0,False,13176,4,0,...,10.3,Mid-Range Product,Regularly busy,Least busy,Average Orders,10,New Customer,6.367797,Low Spender,20.5
9,2398795,1,2,3,7,15.0,False,26088,5,1,...,4.7,Low-Range Product,Regularly busy,Least busy,Average Orders,10,New Customer,6.367797,Low Spender,20.5


In [22]:
# Check the output of the median_days_since_prior_order column
ords_prods_merge['median_days_since_prior_order'].value_counts(dropna = False)

median_days_since_prior_order
7.0     5682399
6.0     2983604
8.0     2713964
5.0     2486061
4.0     2314289
         ...   
26.5       5514
29.5       5095
1.5        1684
0.5         235
NaN           5
Name: count, Length: 62, dtype: int64

In [23]:
# Check why there were 5 NaN values
ords_prods_merge['max_order'].value_counts(dropna = False)

max_order
99    1171333
8      811843
6      811396
9      810213
7      803979
       ...   
97      44949
98      44587
96      40453
2           6
1           5
Name: count, Length: 99, dtype: int64

##### Five customers only made one purchase, so their median_days_since_prior_order would be NaN

### Assign the Frequency Flag to each customer based on the regularity of their ordering behavior

In [24]:
# Assign 'Non-Frequent Customer' to customers with a medain days since prior order of more than 20
ords_prods_merge.loc[ords_prods_merge['median_days_since_prior_order'] > 20, 'frequency_flag'] = 'Non-Frequent Customer'

In [25]:
# Assign 'Regular Customer' to customers with a medain days since prior order of more than 10 and less than or equal to 20
ords_prods_merge.loc[(ords_prods_merge['median_days_since_prior_order'] > 10) & (ords_prods_merge['median_days_since_prior_order'] <= 20), 'frequency_flag'] = 'Regular Customer'

In [26]:
# Assign 'Frequent Customer' to customers with a medain days since prior order of less than or equal to 10
ords_prods_merge.loc[ords_prods_merge['median_days_since_prior_order'] <= 10, 'frequency_flag'] = 'Frequent Customer'

In [27]:
# Check the frequency of the frequency_flag column
ords_prods_merge['frequency_flag'].value_counts(dropna = False)

frequency_flag
Frequent Customer        21559853
Regular Customer          7208564
Non-Frequent Customer     3636437
NaN                             5
Name: count, dtype: int64

##### There are many more Frequent Customers than Non-Frequent or Regular Customers
##### These Frequent Customers would be ideal to target for in-app notifications prompting to repeat their last purchase

In [28]:
# Check the output of the final dataframe with the all of the new columns added
ords_prods_merge.head(10)

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order,new_customer,product_id,add_to_cart_order,reordered,...,price_label,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,average_price_across_orders,spending_flag,median_days_since_prior_order,frequency_flag
0,2539329,1,1,2,8,NaN,True,196,1,0,...,Mid-Range Product,Regularly busy,Regularly busy,Average Orders,10,New Customer,6.367797,Low Spender,20.5,Non-Frequent Customer
1,2539329,1,1,2,8,NaN,True,14084,2,0,...,Mid-Range Product,Regularly busy,Regularly busy,Average Orders,10,New Customer,6.367797,Low Spender,20.5,Non-Frequent Customer
2,2539329,1,1,2,8,NaN,True,12427,3,0,...,Low-Range Product,Regularly busy,Regularly busy,Average Orders,10,New Customer,6.367797,Low Spender,20.5,Non-Frequent Customer
3,2539329,1,1,2,8,NaN,True,26088,4,0,...,Low-Range Product,Regularly busy,Regularly busy,Average Orders,10,New Customer,6.367797,Low Spender,20.5,Non-Frequent Customer
4,2539329,1,1,2,8,NaN,True,26405,5,0,...,Low-Range Product,Regularly busy,Regularly busy,Average Orders,10,New Customer,6.367797,Low Spender,20.5,Non-Frequent Customer
5,2398795,1,2,3,7,15.0,False,196,1,1,...,Mid-Range Product,Regularly busy,Least busy,Average Orders,10,New Customer,6.367797,Low Spender,20.5,Non-Frequent Customer
6,2398795,1,2,3,7,15.0,False,10258,2,0,...,Low-Range Product,Regularly busy,Least busy,Average Orders,10,New Customer,6.367797,Low Spender,20.5,Non-Frequent Customer
7,2398795,1,2,3,7,15.0,False,12427,3,1,...,Low-Range Product,Regularly busy,Least busy,Average Orders,10,New Customer,6.367797,Low Spender,20.5,Non-Frequent Customer
8,2398795,1,2,3,7,15.0,False,13176,4,0,...,Mid-Range Product,Regularly busy,Least busy,Average Orders,10,New Customer,6.367797,Low Spender,20.5,Non-Frequent Customer
9,2398795,1,2,3,7,15.0,False,26088,5,1,...,Low-Range Product,Regularly busy,Least busy,Average Orders,10,New Customer,6.367797,Low Spender,20.5,Non-Frequent Customer


## 08. Exporting the Data

In [29]:
# Export ords_prods_merge to the Prepared Data folder
ords_prods_merge.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_merged_NewVariables_aggregated.pkl'))

In [30]:
ords_prods_merge['prices'].max()

25.0